In [ ]:
import random
import numpy as np
from collections import deque
import flappy_bird_gymnasium
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
env=flappy_bird_gymnasium.FlappyBirdEnv()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
state_size ,action_size

In [ ]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)

In [ ]:
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self):
        experience = random.sample(self.memory, self.batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experience])).float().to(device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experience])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experience])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experience])).float().to(device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experience]).astype(np.uint8)).float().to(device)
        return (states, actions, rewards, next_states,dones)


    def __len__(self):
        return len(self.memory)

In [ ]:
class Agent:
    def __init__(self, state_size, action_size, buffer_size=100000, batch_size=256, gamma=0.999, lr=5e-5, tou=1e-3):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma
        self.lr = lr
        self.tou = tou

        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)
        self.memory = ReplayBuffer(buffer_size, batch_size)
        self.t_steps = 0
        self.update_every = 4

    def act(self, state, epsilon=0.0):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)

        self.qnetwork_local.eval()
        with torch.no_grad():
            q_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
        if random.random() > epsilon:
            return torch.argmax(q_values).item()
        else:
            return np.random.choice(np.arange(self.action_size))

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)
        self.t_steps = (self.t_steps + 1) % self.batch_size
        if self.t_steps % self.update_every == 0 and len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences

        next_actions = self.qnetwork_local(next_states).argmax(1, keepdim=True)
        Q_targets_next = self.qnetwork_target(next_states).gather(1, next_actions).detach()
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        Q_expected = self.qnetwork_local(states).gather(1, actions)
        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.qnetwork_local, self.qnetwork_target)

    def soft_update(self, local_model, target_model):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(target_param.data * (1.0 - self.tou) + local_param.data * self.tou)

In [ ]:
n_episodes = 1000
max_t = 1000
epsilon  = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
scores = []
scores_window = deque(maxlen=100)
agent = Agent(state_size, action_size)

In [ ]:
def shaped_reward(state, next_state, reward, done):
    vertical_dist = state[1]  # Distance from center
    velocity = state[2]       # Current velocity

    # Base rewards
    survival_bonus = 0.2
    alignment_reward = 0.3 * (1 - abs(vertical_dist))

    # Velocity control
    velocity_penalty = -0.2 if velocity < -0.5 else 0.0

    # Pipe passing bonus (detect when we pass a pipe)
    if reward == 1.0:  # Original environment gives +1 for passing pipe
        pipe_bonus = 2.0
    else:
        pipe_bonus = 0.0

    total = (reward + survival_bonus + alignment_reward +
             velocity_penalty + pipe_bonus)

    # Death penalty
    if done and reward == -1:
        total -= 3.0

    return total


In [ ]:
for episode in range(1, n_episodes + 1):
    state, _ = env.reset()
    total_reward = 0

    for t in range(max_t):
        action = agent.act(state, epsilon)
        next_state, reward_env, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        reward = shaped_reward(state, next_state, reward_env, done)
        agent.step(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward

        if done:
            break

    scores.append(total_reward)
    scores_window.append(total_reward)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_window):.2f}', end="")

    if episode % 100 == 0:
        print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_window):.2f}')

    if np.mean(scores_window) >= 15.0:
        print(f'\nEnvironment solved in {episode - 100} episodes!')
        torch.save(agent.qnetwork_local.state_dict(), 'flappy_best.pth')
        break
env.close()


In [ ]:
def record_best_episode(agent, n_trials=20, output_path="best_flappy_video.mp4"):
    best_score = -float('inf')
    best_frames = []
    for trial in range(n_trials):
        env = flappy_bird_gymnasium.FlappyBirdEnv(render_mode="rgb_array")
        state, _ = env.reset()
        frames = []
        score = 0.0
        done = False
        while not done:
            frames.append(env.render())
            action = agent.act(state, epsilon=0.0)  # deterministic evaluation
            next_state, raw_reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            # Use shaped reward for evaluation, matching training:
            score += shaped_reward(state, next_state, raw_reward, done)
            state = next_state
        env.close()
        print(f"Trial {trial+1}: Score = {score}")
        if score > best_score:
            best_score = score
            best_frames = frames

    if best_frames:
        import imageio, os
        imageio.mimsave(output_path, best_frames, fps=30)
        print(f"✅ Best video saved to: {os.path.abspath(output_path)} with score {best_score:.2f}")
    else:
        print("❌ No frames recorded.")

record_best_episode(agent, n_trials=20, output_path="best_flappy_video.mp4")